In [1]:
from BaselineSignals import BaselineSignals 
from Database import Database

In [2]:
from google.cloud import bigquery
import pandas as pd

def get_unique_signals_from_daily_ic(project_id='issachar-feature-library', 
                                     dataset_name='wmg', 
                                     table_name='daily_ic2'):
    """
    Fetch all unique signal values from the 'signal' column in the daily_ic table.
    
    Args:
        project_id (str): The Google Cloud project ID.
        dataset_name (str): The BigQuery dataset name.
        table_name (str): The BigQuery table name containing the daily IC results.
    
    Returns:
        list: A list of unique signal names.
    """
    # Create a BigQuery client.
    client = bigquery.Client(project=project_id)
    
    # Construct the query to select distinct signals.
    query = f"""
    SELECT DISTINCT signal
    FROM `{project_id}.{dataset_name}.{table_name}`
    """
    
    # Execute the query.
    query_job = client.query(query)
    
    # Convert the result to a pandas DataFrame.
    df = query_job.to_dataframe()
    
    # Return the unique signals as a list.
    return df['signal'].unique().tolist()

invalid = ['localPrice', 'lastPrice', 'isin', 'si', 'smi_id', 't1_beta_adjusted_log_returns', 'float', 'invest', 'invest_cap', 'invest_cf', 'isin', 'lastPrice', 'localPrice', 'market_sector_des', 't1_beta_adjusted_log_returns', 'ticker_name', 'universe_start_date', 'sm_id']
all_signals = get_unique_signals_from_daily_ic()
all_signals = list(set( [x for x in all_signals if not x in invalid] ))

In [4]:
from tqdm import tqdm

bs = BaselineSignals()
db = Database()

for s in tqdm(all_signals):
    result = bs.run(s)
    db.get_or_upload_signal(result)
    # print(s, "=>", result)

100%|██████████| 847/847 [30:45<00:00,  2.18s/it]
